In [1]:
# raw data on April 1, 2022 from 16:10 to 16:40 UTC. Launch from SLC40 at 16:24 UTC, watched from Titusville
import os, sys, glob, obspy
import numpy as np
from obspy.signal.trigger import z_detect, classic_sta_lta, plot_trigger, coincidence_trigger
from pprint import pprint
import matplotlib.pyplot as plt
import matplotlib.dates as dates

HOME = os.getenv('HOME')
SDS_TOP = os.path.join(HOME, 'Desktop/Desktop - Glenn’s MacBook Air/KSC_new')
DROPBOX_TOP = os.path.join(HOME, 'Dropbox')
DROPBOX_PROJECT_DIR = os.path.join(DROPBOX_TOP, 'PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion')
EVENT_MSEED_DIR = os.path.join(DROPBOX_PROJECT_DIR, 'KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events')
HTML_DIR = './html'
PLOTS_DIR = HTML_DIR

launch_time_strings = ["2022-04-01T16:24:00", 
                       "2022-04-08T15:17:00", 
                       "2022-04-21T17:51:00", 
                       "2022-04-27T07:52:00", 
                       "2022-04-29T09:27:00", 
                       "2022-05-06T09:52:00",
                       "2022-05-14T08:40:00",
                       "2022-05-18T10:59:00",
                       "2022-05-19T22:54:00",
                       "2022-05-25T18:35:00",
#                       "2022-06-07T21:03:00",
#                       "2022-06-09T14:45:00"
                      ]

def time_TO_miniseed_filename(launchtime, raw=True, calibrated=False, cleaned=False):
    mseedfilename = os.path.join(EVENT_MSEED_DIR, 'launch_%s.mseed' % launchtime.strftime('%Y%m%dT%H%M%S') )
    if raw:
        mseedfilename = mseedfilename.replace('.mseed', '_raw.mseed')
    if cleaned:
        mseedfilename = mseedfilename.replace('.mseed', '_clean.mseed')
    if calibrated:
        mseedfilename = mseedfilename.replace('.mseed', '_calibrated.mseed')        
    return mseedfilename

def sds_TO_event(launch_time_strings):
    pretrig = 600
    posttrig = 1200
    from obspy.clients.filesystem.sds import Client
    client = Client(SDS_TOP, sds_type='D', format='mseed')
    for this_lts in launch_time_strings:
        launchtime = obspy.UTCDateTime(this_lts)
        mseedfile = time_TO_miniseed_filename(launchtime)
        if os.path.exists(mseedfile):
            print('%s already exists. Loading.' % mseedfile)
        else:
            print('%s not found. Segmenting from SDS.' % mseedfile)        
            st = client.get_waveforms('FL', '*', '*', 'H*', launchtime - pretrig, launchtime + posttrig, merge=-1)
            st.write(mseedfile, 'mseed')

def load_event(launchtime):            
    mseedfile = time_TO_miniseed_filename(launchtime)
    st = obspy.read(mseedfile)    
    st.merge()
    print('%s: %d channels' % (mseedfile,len(st)))
    return st

def clean(st):
    for tr in st:
        tr.detrend('linear')
        tr.filter('highpass', freq=0.01, corners=2)    
        
def manually_select_good_traces(st):
    straw = obspy.Stream()
    for tr in st:
        tr.detrend('linear')
        tr.filter('highpass', freq=0.01, corners=2)
        tr.plot()
        yn = input('include?')
        if yn.lower() == 'y':
            straw.append(tr)
    return straw

def look_for_signal(st, launchtime, threshold=1.5):
    st2 = obspy.Stream()
    clean(st)
    for tr in st:
        tr_noise = tr.copy().trim(starttime=launchtime-240, endtime=launchtime-120)
        tr_signal = tr.copy().trim(starttime=launchtime, endtime=launchtime+120)
        avnoise = np.nanmedian(np.abs(tr_noise.data))
        avsignal = np.nanmedian(np.abs(tr_signal.data))
        #print(avsignal, avnoise)
        if avsignal > threshold * avnoise:
            st2.append(tr)
    return st2

def detectEvent(st, launchtime):
    trig = coincidence_trigger("recstalta", 3.5, 1, st2, 3, sta=2, lta=40)
    best_trig = {}
    best_product = 0
    for this_trig in trig:
        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    st_detected = obspy.Stream()
    for tr in st:
        if tr.id in best_trig['trace_ids']:
            st_detected.append(tr)
    #if len(st_detected)>0:
    #    st_detected.plot(equal_scale=False, starttime=best_trig['time']-60, endtime=best_trig['time']+120 )
    #input("ENTER to continue")
    return st_detected, best_trig['time']


def apply_calibration_correction(straw):
    # calibration correction
    stcorrected = straw.copy()
    for tr in stcorrected:
        if 'countsPerUnit' in tr.stats:
            continue
        else:
            tr.stats['countsPerUnit'] = 0.0  
            if tr.stats.channel[1]=='D':
                tr.stats.countsPerUnit = 720 # counts/Pa on 1 V FS setting
                if tr.id[:-1] == 'FL.BCHH3.10.HD':
                    if tr.stats.starttime < obspy.UTCDateTime(2022,5,26): # Chaparral M25. I had it set to 1 V FS. Should have used 40 V FS. 
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 8e5 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 720 # counts/Pa 
                    else: # Chaparral switched to 40 V FS
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 2e4 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 18 # counts/Pa 
                tr.stats.units = 'Pa'

            elif tr.stats.channel[1]=='H':
                tr.stats.countsPerUnit = 3e8 # counts/(m/s)
                tr.stats.units = 'm/s'
            tr.data = tr.data/tr.stats.countsPerUnit
        #print(tr.id, tr.stats.countsPerUnit)
    return stcorrected        

def maxamp(tr):
    return np.max(np.abs(tr.data))

def remove_spikes(st):
    SEISMIC_MAX = 0.1 # m/s
    INFRASOUND_MAX = 3000 # Pa
    SEISMIC_MIN = 1e-8
    INFRASOUND_MIN = 0.01
    st2 = obspy.Stream()
    
    for tr in st:
        ma = maxamp(tr)
        if tr.stats.units == 'm/s':
            if ma < SEISMIC_MAX and ma > SEISMIC_MIN:
                st2.append(tr)
            else:
                print('Not appending %s. Max value %e m/s out of range' % (tr.id, ma))               
        elif tr.stats.units == 'Pa':
            if ma < INFRASOUND_MAX  and ma > INFRASOUND_MIN:
                st2.append(tr)    
            else:
                print('Not appending %s. Max value %e Pa out of range' % (tr.id, ma))
    return st2           

# ********** MAIN PROGRAM **********
sds_TO_event(launch_time_strings)
real_launch_times = []
for this_lts in launch_time_strings:
    launchtime = obspy.UTCDateTime(this_lts)
    print('Processing launch at %s' % launchtime.strftime('%Y-%m-%d %H:%M:%S'))
    st = load_event(launchtime)
    print(st)
    
    clean(st)
    print('- %d channels after cleaning' % len(st))
    
    stc = apply_calibration_correction(st)
    print('- %d channels after calibrating' % len(stc))
    
    #st2 = stc.copy()
    # SCAFFOLD: alter this to applied thresholds
    st2 = remove_spikes(stc)
    print('- %d channels after despiking' % len(st2))
    
    # SCAFFOLD: alter this to detect events & correct launchtime
    st3, reallaunchtime = detectEvent(st2, launchtime)
    real_launch_times.append(reallaunchtime)
    print('- %d channels after detection' % len(st3))
    
    st4 = look_for_signal(st3, reallaunchtime, threshold=1.1)
    print('- %d channels after SNR check' % len(st4))
    
    if len(st4)>0:
        mseedfile = time_TO_miniseed_filename(reallaunchtime, raw=False, cleaned=True, calibrated=True)
        print('Writing %s' % mseedfile)
        st4.write(mseedfile)
        
print(real_launch_times)

/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220401T162400_raw.mseed already exists. Loading.
/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220408T151700_raw.mseed already exists. Loading.
/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220421T175100_raw.mseed already exists. Loading.
/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220427T075200_raw.mseed already exists. Loading.
/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC

/Users/thompsong/miniconda3/envs/AAA/lib/python3.8/site-packages/obspy/io/mseed/core.py:790: UserWarning: The encoding specified in trace.stats.mseed.encoding does not match the dtype of the data.
A suitable encoding will be chosen.
  warnings.warn(msg, UserWarning)


/Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220408T151700_raw.mseed: 24 channels
24 Trace(s) in Stream:

FL.BCHH2.10.HD4 | 2022-04-08T15:07:00.000000Z - 2022-04-08T15:37:00.000000Z | 100.0 Hz, 180001 samples
...
(22 other traces)
...
FL.S39A3.10.HDF | 2022-04-08T15:07:00.000000Z - 2022-04-08T15:37:00.000000Z | 100.0 Hz, 180001 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]
- 24 channels after cleaning
- 24 channels after calibrating
Not appending FL.BCHH2.10.HD5. Max value 6.464645e-12 Pa out of range
- 23 channels after despiking
{'coincidence_sum': 16.0,
 'duration': 94.91000008583069,
 'similarity': {},
 'stations': ['S39A3',
              'S39A1',
              'BCHH3',
              'S39A2',
              'S39A2',
              'S39A2',
              'S39A1',
              'S39A3',
              'S39A1',
              'S39A1',


- 24 channels after cleaning
- 24 channels after calibrating
- 24 channels after despiking
{'coincidence_sum': 9.0,
 'duration': 42.69000005722046,
 'similarity': {},
 'stations': ['BCHH2',
              'S39A2',
              'S39A2',
              'S39A2',
              'S39A1',
              'BCHH2',
              'BCHH3',
              'BCHH3',
              'BCHH3'],
 'time': UTCDateTime(2022, 5, 6, 9, 56, 44, 140000),
 'trace_ids': ['FL.BCHH2.10.HD6',
               'FL.S39A2.00.HHN',
               'FL.S39A2.00.HHE',
               'FL.S39A2.00.HHZ',
               'FL.S39A1.00.HHZ',
               'FL.BCHH2.10.HD5',
               'FL.BCHH3.00.HHZ',
               'FL.BCHH3.00.HHN',
               'FL.BCHH3.00.HHE']}
- 9 channels after detection
- 5 channels after SNR check
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220506T095644_clean_calibr

- 17 channels after SNR check
Writing /Users/thompsong/Dropbox/PROFESSIONAL/RESEARCH/3 Project Documents/NASAprojects/202010 KSC Launchpad Erosion/KSC_Well_Seismoacoustic_Data/SeismoAcousticData/Events/launch_20220525T183500_clean_calibrated.mseed
[UTCDateTime(2022, 4, 1, 16, 24, 19, 820000), UTCDateTime(2022, 4, 8, 15, 16, 42, 80000), UTCDateTime(2022, 4, 21, 17, 47, 28, 70000), UTCDateTime(2022, 4, 27, 7, 52, 39, 270000), UTCDateTime(2022, 4, 29, 9, 27, 41, 690000), UTCDateTime(2022, 5, 6, 9, 56, 44, 140000), UTCDateTime(2022, 5, 14, 8, 42, 26, 540000), UTCDateTime(2022, 5, 18, 10, 59, 24, 70000), UTCDateTime(2022, 5, 19, 22, 54, 42, 300000), UTCDateTime(2022, 5, 25, 18, 35, 0, 540000)]
